In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from modules import *
from astropy.table import Table
import scipy.signal as signal
import scipy.optimize as optimize
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from tqdm import tqdm

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 219107776'

In [ ]:
lcs = get_lightcurves(TIC, author = 'SPOC', cadence='short', use_till=10)

In [ ]:
new_lcs = []

for lc in lcs:
    df = pd.DataFrame({'time': lc.time.jd, 'flux': np.array(lc.flux, dtype='f')}).dropna(inplace=False)
    new_lc = lk.LightCurve(time= df['time'], flux= df['flux'])
    new_lc = apply_savgol_filter(new_lc.time.jd, new_lc.flux, want = 'lc', displaygraphs= False, window_length_for_remove=1500)
    new_lc = straight_lines(new_lc, cadence_magnifier= 10)
    new_lcs.append(new_lc)
    # new_lc.plot()

In [ ]:
lightcurve = combine_lightcurves(new_lcs)
lightcurve.plot()

In [ ]:
new_lightcurve = straight_lines(lightcurve, cadence_magnifier= 1)
axess = new_lightcurve.plot()
lightcurve.plot(ax= axess, color= 'red')

print(pd.DataFrame({'time': new_lightcurve.time.jd, 'flux': new_lightcurve.flux}).info())
print(pd.DataFrame({'time': lightcurve.time.jd, 'flux': lightcurve.flux}).info())


FINDING THE PERIOD
-------------

The next section here finds the period to be used for O-C calculations as well as for folding.
A gaussian curve is fit on a Periodogram that is obtained from the desired `lightcurve`.

Please click on the desired peak to make the program process data, the peak should be the tallest peak in the entire mess of principle peak.

PS: Clicking on a point draws a gaussian which is selected for the O-C calculations. If you missed_click, just click on the right point afterwards and it will draw a curve in a different color, allowing you to choose another point.

In [ ]:
periodogram = lightcurve.to_periodogram(maximum_frequency = 30)

peak_width = 1 / ( (lightcurve.time.btjd[-1] - lightcurve.time.btjd[0]) * 2)
print(peak_width)
max_power = periodogram.max_power.value
optimized_parameters_global = []

colors = ['red', 'green', 'purple', 'orange', 'yellow', 'pink', 'brown', 'magenta', 'cyan']
def color_change():
    c = colors.pop(0)
    colors.append(c)
    yield c

frequencies = periodogram.frequency.value
power = periodogram.power.value
f = np.linspace(frequencies.min(), frequencies.max(), 1000000)
p = spline(frequencies, power, k = 3)(f)

peaks, peakdict = signal.find_peaks(p, height = max_power/10)
peaks_df = pd.DataFrame({'frequencies': f[peaks], 'power': p[peaks], 'peak_index': peaks})

fi = go.FigureWidget([
    go.Scatter(x=f[peaks], y=p[peaks], mode = 'markers', name = 'Peaks', marker = {
        'symbol': 'x',
        'size': 7,
    }),
    go.Scatter(x=frequencies, y=power, mode='lines', name = 'Periodogram', line = dict(color='black')),
], layout = go.Layout(title = 'Periodogram Peaks', xaxis_title = 'Frequency [1/d]', yaxis_title = 'Power'))

scatter = fi.data[0]
fi.layout.hovermode = 'closest'

def update_point(trace, points, selector):
    for i in points.point_inds:
        initial_guess = [peaks_df['power'][i], peaks_df['frequencies'][i], peak_width]
        optimized_parameters, covariance = optimize.curve_fit(gaussian, frequencies, power, p0=initial_guess)
        amp, cen, wid = optimized_parameters

        fitted_gaussian = gaussian(frequencies, amp, cen, wid)
        initial_gaussian = gaussian(frequencies, *initial_guess)

        fi.add_trace(go.Scatter(x=frequencies, y=fitted_gaussian, mode='lines', name='Fitted Gaussian', line=dict(color=next(color_change()))))
        fi.add_trace(go.Scatter(x=frequencies, y=initial_gaussian, mode='lines', name='Initial Gaussian', line=dict(color=next(color_change()))))
        optimized_parameters_global.append(optimized_parameters)
        print(optimized_parameters_global[-1])
    
scatter.on_click(update_point)
fi


In [ ]:
try:
    fitted_frequency = optimized_parameters_global[-1][1]
    fitted_period = 1/fitted_frequency
except:
    raise ValueError('No peaks clicked yet, please click on a peak to fit a gaussian to it.')

print(f'Fitted Frequency: {fitted_frequency}')
print(f'Fitted Period: {fitted_period}')

In [ ]:
lightcurve_df = pd.DataFrame({'time': lightcurve.time.btjd, 'flux': np.array(lightcurve.flux, dtype='f')})
curr_initial_time = lightcurve_df['time'].iloc[0]
curr_end_time = curr_initial_time + fitted_period

chunks = []

for i in tqdm(range(2000)):
    if curr_end_time > lightcurve_df['time'].iloc[-1]:
        break
    else:
        chunk = lightcurve_df[(lightcurve_df['time'] >= curr_initial_time) & (lightcurve_df['time'] <= curr_end_time)]
        curr_initial_time = curr_end_time
        curr_end_time = curr_initial_time + fitted_period
        plt.figure(num = 1, figsize=(10, 5))
        plt.plot(chunk['time'], chunk['flux'], lw=0.5)
        # plt.savefig(f'Chunks/{i}.png')
        chunks.append(chunk)


In [ ]:
l = get_lightcurves(TIC, author = 'SPOC', cadence='short', use_till=2)
l[0].fold(fitted_period).bin(time_bin_size = 0.0005).scatter()